Using ANN, SVR and RFR to estimate seagrass percentage coverage

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

root_dir = '/content/gdrive/My Drive/'

In [ ]:
from tensorflow.keras import models, layers, utils, backend as K

import matplotlib.pyplot as plt

import shap

import numpy as np

from sklearn.model_selection import train_test_split

from keras import models

from keras import layers

import tensorflow as tf

import cv2

import xlrd

In [ ]:
table = xlrd.open_workbook('/content/gdrive/My Drive/seagrass_coverage/original_data.xlsx')

sheet = table.sheets()[0]

x1 = sheet.col_values(6,1) #Band Blue

x2 = sheet.col_values(7,1) #Band Green

x3 = sheet.col_values(8,1) #Band Red

x4 = sheet.col_values(9,1) #Band NIR

x5 = sheet.col_values(10,1) #NDWI

x6 = sheet.col_values(11,1) #NDVI

x = np.array([x1,x2,x3,x4,x5,x6]).T #Transpose the array

y = np.array(sheet.col_values(5,1))/100 #seagrass percentage coverage

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2) #Split the data; 80% for training

x_test, x_val, y_test, y_val = train_test_split(x_test,y_test,test_size = 0.5) #10% for validation and 10% for testing

In [ ]:
from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score

import math

result = []

for m in range(2,40,2):
  tf.keras.backend.clear_session()
  model = models.Sequential()
  tf.random.set_seed(1)
  model.add(layers.Dense(m, activation = 'relu', input_shape = (x_train.shape[1], ))) ##Change different activation functions to find out the best one.
  model.add(layers.Dense(m, activation = 'relu')),
  model.add(layers.Dense(1,))
  opt = tf.keras.optimizers.Adam(learning_rate=0.01) ##Change different Learning rate to find out the best one.
  model.compile(opt, loss = 'mse', metrics = ['mse'])
  model.fit(x_train,y_train,epochs = 1000,batch_size = 300, verbose = 0)
  y_val_test = model.predict(x_val)
  r2 = r2_score(y_val,y_val_test)

#model.summary()

#The optimal values:

#Number of hidden layers: 2 [Please check the methods in the paper]

#Number of neurons : 10

#Activation function: ReLu

#Learning rate: 0.01

In [ ]:
tf.random.set_seed(1)

model = models.Sequential()

tf.keras.backend.clear_session()

model.add(layers.Dense(10, activation = 'relu', input_shape = (x_train.shape[1], )))

model.add(layers.Dense(10,activation = 'relu')),

model.add(layers.Dense(1,))

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(opt, loss = 'mse', metrics = ['mse'])

reuslt = model.fit(x_train,y_train,epochs = 1000,batch_size = 300, verbose = 0)

y_test_predict = model.predict(x_test)

plt.figure(figsize=[5,5])

plt.scatter(y_test,y_test_predict)

xx = [0,1]

yy = [0,1]

plt.plot(xx,yy,'-')

plt.xlim([0,1])

plt.ylim([0,1])

from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score

import math

print(math.sqrt(mean_squared_error(y_test_predict.flatten(),y_test.flatten())))

# 0.11175243324504268

print(r2_score(y_test_predict,y_test))

# 0.7106385173472587